# Praca Domowa 6
Bartosz Siński

In [ ]:
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
from sklearn.datasets import fetch_olivetti_faces
np.set_seed = 42

## Załadowanie danych i rysowanie obrazów

In [ ]:
faces, _ = fetch_olivetti_faces(return_X_y=True, shuffle=True,
                                random_state=1613)

In [ ]:
faces.shape

In [ ]:
images_id = [355,145,267,289,375,1]
imshape = (64,64)

In [ ]:
fig, axs = plt.subplots(ncols=3,nrows=2,figsize=(15,10))
for i in range(3):
    axs[0,i].imshow(faces[images_id[i]].reshape(imshape),vmin=0, vmax=1)
    axs[0,i].set_title(i+1)
    axs[1,i].imshow(faces[images_id[3+i]].reshape(imshape),vmin=0, vmax=1)
    axs[1,i].set_title(4+i)

## Przygotowanie PCA i dobranie liczby składowych.

Nasze wartości znajduja się w przediale [0,1] więc nie ma potrzeby ich skalowania.

In [ ]:
from sklearn.decomposition import PCA
pca = PCA().fit(faces)
variance = np.cumsum(pca.explained_variance_ratio_)[0:400]
plt.plot(range(400),variance)
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance ratio')

In [ ]:
np.cumsum(pca.explained_variance_ratio_)[199]

Widzimy, że przy pozostawieniu 200 komponentów składowych mamy jeszcze zachowane prawie 98% wariancji więc wybierzemy właśnie te liczby komponentów.

In [ ]:
pca200 = PCA(n_components=200).fit(faces)
faces_reduced = pca200.transform(faces)
print('Stopień kompresji: '+ str(faces.shape[1]/faces_reduced.shape[1]))

In [ ]:
faces_pca = pca200.inverse_transform(faces_reduced)
fig, axs = plt.subplots(ncols=3,nrows=2,figsize=(15,10))
for i in range(3):
    axs[0,i].imshow(faces_pca[images_id[i]].reshape(imshape),vmin=0, vmax=1)
    axs[0,i].set_title(i+1)
    axs[1,i].imshow(faces_pca[images_id[3+i]].reshape(imshape),vmin=0, vmax=1)
    axs[1,i].set_title(i+4)

In [ ]:
from sklearn.metrics import mean_squared_error
for i in range(6):
    print('Błąd rekonstrukcji w postaci RMSE dla obrazka ' + str(i+1) +" = " + str(mean_squared_error(faces[images_id[i]],faces_pca[images_id[i]])))
print('Średni błąd rekonstrukcji w postaci RMSE dla wybranych obrazków: '+ str(mean_squared_error(faces[images_id],faces_pca[images_id])))  

Przede wszystkim zdjęcia wydają się być mniej ostre. Dodatkowo wsród osób bez okularów pojawiły się obramowania wokół oczu. Najmniejszy błąd przy rekonstrukcji był dla zdjęcia 3, na którym znajduje się kobieta o gładkich rysach bez okularów, a największy dla zdjęcia 4 gdzie znajduje się starszy pan w okularach.

## Zmodyfikowane obrazy na wcześniej wyuczonym modelu
### Obrazy obrócone 

In [ ]:
order = -np.sort(-np.arange(4096))
faces_reversed = faces[:,order]
fig, axs = plt.subplots(ncols=3,nrows=2,figsize=(15,10))
for i in range(3):
    axs[0,i].imshow(faces_reversed[images_id[i]].reshape(imshape),vmin=0, vmax=1)
    axs[0,i].set_title(i+1)
    axs[1,i].imshow(faces_reversed[images_id[3+i]].reshape(imshape),vmin=0, vmax=1)
    axs[1,i].set_title(i+4)

In [ ]:
faces_r = pca200.transform(faces_reversed)
faces_r = pca200.inverse_transform(faces_r)
fig, axs = plt.subplots(ncols=3,nrows=2,figsize=(15,10))
for i in range(3):
    axs[0,i].imshow(faces_r[images_id[i]].reshape(imshape),vmin=0, vmax=1)
    axs[0,i].set_title(i+1)
    axs[1,i].imshow(faces_r[images_id[3+i]].reshape(imshape),vmin=0, vmax=1)
    axs[1,i].set_title(i+4)

In [ ]:
print('Błąd rekonstrukcji w postaci RMSE dla wybranych obrazków: '+ str(mean_squared_error(faces_reversed[images_id],faces_r[images_id])))

Z PCA nauczonym na normalnych obrazach, odwrócone obrazy po rekonstrukcji w żaden sposób nie przypominają oryginałów. Widać za to jeszcze bardziej "dorysowywanie" okularów przy rekonstrukcji. Błędy RMSE są także ponad 10 razy większe niz poprzednio.

## Obrazy odwrócone w poziomie

In [ ]:
order = np.arange(0,4096)
order = np.split(order,64)
def fun(arr):
    return(-np.sort(-arr))
order = np.apply_along_axis(fun, 1, order)
order = np.reshape(order,(1,4096))[0]
faces_reversed2 = faces[:,order]
fig, axs = plt.subplots(ncols=3,nrows=2,figsize=(15,10))
for i in range(3):
    axs[0,i].imshow(faces_reversed2[images_id[i]].reshape(imshape),vmin=0, vmax=1)
    axs[0,i].set_title(i+1)
    axs[1,i].imshow(faces_reversed2[images_id[3+i]].reshape(imshape),vmin=0, vmax=1)
    axs[1,i].set_title(i+4)

In [ ]:
faces_r2 = pca200.transform(faces_reversed2)
faces_r2 = pca200.inverse_transform(faces_r2)
fig, axs = plt.subplots(ncols=3,nrows=2,figsize=(15,10))
for i in range(3):
    axs[0,i].imshow(faces_r2[images_id[i]].reshape(imshape),vmin=0, vmax=1)
    axs[0,i].set_title(i+1)
    axs[1,i].imshow(faces_r2[images_id[3+i]].reshape(imshape),vmin=0, vmax=1)
    axs[1,i].set_title(i+4)

In [ ]:
print('Błąd rekonstrukcji w postaci RMSE dla wybranych obrazków: '+ str(mean_squared_error(faces_reversed2[images_id],faces_r2[images_id])))

Błędy rekonstrukcji są mniejsze niż w przypadków obrazów obróconych o 180 stopni jednak nadal są ponad 5 razy większe niż w przypadku oryginalnych zdjęć. Obrazy mocno różnią się od oryginałów.

### Obrazy przyciemnione

In [ ]:
faces_dimmed = faces - 0.2
fig, axs = plt.subplots(ncols=3,nrows=2,figsize=(15,10))
for i in range(3):
    axs[0,i].imshow(faces_dimmed[images_id[i]].reshape(imshape),vmin=0, vmax=1)
    axs[0,i].set_title(i+1)
    axs[1,i].imshow(faces_dimmed[images_id[3+i]].reshape(imshape),vmin=0, vmax=1)
    axs[1,i].set_title(i+4)

In [ ]:
faces_d = pca200.transform(faces_dimmed)
faces_d = pca200.inverse_transform(faces_d)
fig, axs = plt.subplots(ncols=3,nrows=2,figsize=(15,10))
for i in range(3):
    axs[0,i].imshow(faces_d[images_id[i]].reshape(imshape),vmin=0, vmax=1)
    axs[0,i].set_title(i+1)
    axs[1,i].imshow(faces_d[images_id[3+i]].reshape(imshape),vmin=0, vmax=1)
    axs[1,i].set_title(i+4)

In [ ]:
print('Błąd rekonstrukcji w postaci RMSE dla wybranych obrazków: '+ str(mean_squared_error(faces_dimmed[images_id],faces_d[images_id])))

### Obrazy rozjaśnione

In [ ]:
faces_bright = faces + 0.2
fig, axs = plt.subplots(ncols=3,nrows=2,figsize=(15,10))
for i in range(3):
    axs[0,i].imshow(faces_bright[images_id[i]].reshape(imshape),vmin=0, vmax=1)
    axs[0,i].set_title(i+1)
    axs[1,i].imshow(faces_bright[images_id[3+i]].reshape(imshape),vmin=0, vmax=1)
    axs[1,i].set_title(i+4)

In [ ]:
faces_b = pca200.transform(faces_bright)
faces_b = pca200.inverse_transform(faces_b)
fig, axs = plt.subplots(ncols=3,nrows=2,figsize=(15,10))
for i in range(3):
    axs[0,i].imshow(faces_b[images_id[i]].reshape(imshape),vmin=0, vmax=1)
    axs[0,i].set_title(i+1)
    axs[1,i].imshow(faces_b[images_id[3+i]].reshape(imshape),vmin=0, vmax=1)
    axs[1,i].set_title(i+4)

In [ ]:
print('Błąd rekonstrukcji w postaci RMSE dla wybranych obrazków: '+ str(mean_squared_error(faces_bright[images_id],faces_b[images_id])))

Zarówno przyciemnone i jak i rozjaśnione obrazy zostały zrekonstruowane z podobną jakością jak zdjęcia oryginalne.

## Obrazy z  odwróconymi kolorami

In [ ]:
faces_color = -faces 
fig, axs = plt.subplots(ncols=3,nrows=2,figsize=(15,10))
for i in range(3):
    axs[0,i].imshow(faces_color[images_id[i]].reshape(imshape),vmin=-1, vmax=0)
    axs[0,i].set_title(i+1)
    axs[1,i].imshow(faces_color[images_id[3+i]].reshape(imshape),vmin=-1, vmax=0)
    axs[1,i].set_title(i+4)

In [ ]:
faces_c = pca200.transform(faces_color)
faces_c = pca200.inverse_transform(faces_c)
fig, axs = plt.subplots(ncols=3,nrows=2,figsize=(15,10))
for i in range(3):
    axs[0,i].imshow(faces_c[images_id[i]].reshape(imshape),vmin=-1, vmax=0)
    axs[0,i].set_title(i+1)
    axs[1,i].imshow(faces_c[images_id[3+i]].reshape(imshape),vmin=-1, vmax=0)
    axs[1,i].set_title(i+4)

In [ ]:
print('Błąd rekonstrukcji w postaci RMSE dla wybranych obrazków: '+ str(mean_squared_error(faces_color[images_id],faces_c[images_id])))

Odwrócenie kolorów zdecydowanie obniżyło jakość rekonstrukcji obrazów.

### Wnioski
Rezultaty transformacji odwrotnej obrazów z 200 komponentów do 4096 okazały się bardzo zadowalające. W przypadku większości obrazów oryginalnych, do których fitowaliśmy PCA, ciężko było znaleźć różnice pomiędzy oryginałami, a zrekonstruowanymi obrazami. Zaskakujące okazało się to, że obrazy pociemnione i rozjaśnione zostały zrekonstruowane prawie tak samo dobrze jak oryginały. W przypadku bardziej zniekształcających obraz transformacji (np. obrót obrazu o 180 stopni) wyniki rekonstrukcji były zdecydowanie gorsze. Nasza metoda odtwarzania była wrażliwa na okulary i na większości obrazów je dorysowywała. PCA mogłoby więc posłużyć do określenia w prosty sposób czy wśród naszych obrazów są zdjęcia ludzi w okularach. 